# Requests

`requests` is a python HTTP library that allows you to send HTTP/1.1 requests.

In [ ]:
import requests

res = requests.get("https://google.ca")
print(res.status_code)
print(res.content)

# Bottle

Run the following to specify ports that `bottle` will listen from.

Note:
There is a `run` command at the end of each subsequent cell, which causes `bottle` to listen to HTTP requests indefinitely.
As you can only have one `bottle` server (to the same port) at a time, stop the previous cells before running the next one.

In [ ]:
PORT=8081

## Routes

The `route` decorator can be used to hook the url to the function.
In the example below, whenever there is a `GET` request to `localhost:8081/`, `bottle` will call `home` function, and use the returned value as a response.

In [ ]:
from bottle import run, route

@route("/")  # The route decorator serves as a hook for linking / to home()
def home():
    return '<html><body> Hello! </body></html>'

run(host='localhost', port=PORT)

Run the above command, and go to `localhost:8081` from a web browser, you will see a message `Hello!` on the screen.

### Wildcards

You can include wildcards as part of the route, which is of the form `<variable name>`.
The wildcards match any (non-empty) strings.
When the callback is used, the variable with the same name is populated with the matched string.

In [ ]:
from bottle import run, route

@route("/home/<name>")  # Dynamic route, matches /home/Alice, /home/Bob, etc.
def home2(name):
    return f"<html><body> Hello {name}! </body></html>"

run(host='localhost', port=PORT)

Go to `localhost:8081/home/someone`, and you will see `Hello someone!` on the screen.
If you go to `localhost:8081/home`, you will receive a 404 error because it **DOES NOT** match `/home/<name>`.

Note:
The variable `name` is created and used only on the server side.
The browser only receives the interpolated string `Hello someone!`.
You can verify by inspecting the website's source code from your web browser.

### Multiple Routes with the Same Callback

You can have multiple routes hooked up to the same callback.
If the route does not have a wildcard, its default value (which must be provided) is used.

In [ ]:
from bottle import run, route

@route("/greeting")
@route("/home2/<name>")
# Multiple routes can be mapped to the same callback function
def greeting(name="Stranger"):
    return f"<html><body> Hello {name}! </body></html>"

run(host='localhost', port=PORT)

### Filtered Wildcards

You can filter wildcards using `<variable name:filter>`.
The following uses an `re:` filter on wildcard `filename` so that it only matches a regex `.*\.html`.

In [ ]:
from bottle import run, route, static_file

@route("/static/<filename:re:.*\\.html>")  # regex to match any .html file
def static(filename):
    # Read file from the static/ subdirectory
    return static_file(filename, root='static')

run(host='localhost', port=PORT)

If you go to `localhost:8081/static/abcd`, you will receive a 404 error because it does not match the regex specified for the `static` route.
However, if you go to `localhost:8081/static/sample.html`, `bottle` will response with the file content in `/static/sample.html`.

### Handling HTML Forms

Forms can be created using `<form>` tag.
We commonly uses `POST` or `PUT` methods for the form action (see `form` tag attribute in `static.html/static`).
In this case, we can handle a `POST` method to `/login` (per `form` tag attributes) using `@post` decorator, as shown in `process_login`.

You can use `request.forms` to retrieve the form's content.

In [ ]:
from bottle import run, get, post, request

@get("/login")  # This is equivalent to @route("/login")
def login():
    return static_file('login.html', root='static')

@post("/login")
def process_login():
    # Process the form data
    username = request.forms.get('username')
    password = request.forms.get('password')
    # Echo back the username and password
    return f"<html><body> Hello {username}! PW:{password} </body></html>"

run(host='localhost', port=PORT)

Go to `localhost:8081/login`, and submit the form.
You should see the hello message with appropriate username (and password)!

Note:
As shown here, the form includes the inputs (including password inputs) in plain text.
Websites generally employ additional processing to ensure the inputs are sent securely.

#### (Optional) Using `routes` to Submit a Form

You can also use `requests` to submit the data to the form.
This is equivalent to pressing login on the form earlier.

Note: since this requires the previous cell to finish, you need to run bottle using `python3 tut2.py` instead of this notebook directly.
Make sure to run the command from the the directory where `tut2.py` is in.

In [ ]:
from requests import post

res = post("http://localhost:8081/login", data={"username": "Alice", "password": "Wonderland"})
print(res.status_code)
print(res.content)

### Errors and Redirects

You can use `abort` to return an HTML error with custom messages, and use `redirect` to redirect the website to another directory.

In [ ]:
from bottle import run, route, abort, redirect

@route("/restricted")
def restricted():
    abort(401, "You are not allowed to access this page.")

@route("/wrong")
def wrong():
    redirect("/")  # Go back to home page

run(host='localhost', port=PORT)

- Go to `localhost:8081/restricted`. You should receive a 401 error (with the same message as specified in `abort`).
- Go to `localhost:8081/wrong`. You should be redirected to `localhost:8081/`. You can verify this by checking the page address on your web browser.

### Cookies

You can use `request.get_cookie` and `response.set_cookie` to retrieve and assign cookies.
By default, the cookies are per-session, meaning that they will be deleted when the browser closes.

In [ ]:
from bottle import run, route, request, response

@route("/cookie")
def hello_cookie():
    if request.get_cookie("visited"):
        return f"Welcome back! Nice to see you again (cookies = {request.get_cookie("visited")})"
    else:
        response.set_cookie("visited", "yes")
        return "Hello! Nice to meet you!"

run(host='localhost', port=PORT)

Visit `localhost:8081/cookie`, you should see *"Hello! Nice to meet you"*.
If you visit `localhost:8081/cookie` again, you should now see *"Welcome back! Nice to see you again (cookies = yes)"*.

### Templates

You can use templates, similar to template strings in python.
A template in `bottle` uses `{{variable name}}` to specify the placeholder.
You can then use `template` function to replace placeholders with appropriate values.

The `template` function also accepts a file path instead of raw strings.

In [ ]:
from bottle import run, route, template

@route("/template")
def template_demo():
    # You can pass variables as keyword arguments
    tpl = template("Hello {{name}}", name="World")
    # You can also unpack a dictionary into keyword arguments
    dict = {"dept": "ECE", "code": "ECE326"}
    tpl2 = template("Department {{dept}} Course Code:{{code}}", **dict)
    return tpl + '</br>' + tpl2


@route("/template2")
def template_demo2():
    basket_list = ["A", "B", "C"]
    # You can also specify the template file directly
    return template("tpl.html", basket=basket_list)

run(host='localhost', port=PORT)